In [1]:
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv("train_sample.csv")
test_data = pd.read_csv("test_sample.csv")


In [7]:
data

,X1,X2,X3,X4,X5,X6,X7,class
0,0.217707,0.634695,0.028377,0.093866,0.170345,0.218078,0.254370,0
1,0.976832,0.196437,0.593828,0.225760,0.810395,0.215981,0.463485,1
2,0.474112,0.867036,0.786070,0.634799,0.562943,0.470834,0.642650,1
3,0.435562,0.559760,0.791351,0.891573,0.853098,0.609158,0.281369,1
4,0.497564,0.188632,0.208090,0.960607,0.935006,0.767656,0.707381,1
...,...,...,...,...,...,...,...,...
495,0.843427,0.707178,0.868424,0.785170,0.103390,0.628541,0.669493,1
496,0.922892,0.375219,0.740096,0.045435,0.638138,0.207538,0.697541,0
497,0.548765,0.478041,0.862437,0.822143,0.639047,0.554049,0.857795,1
498,0.251716,0.348931,0.817141,0.137908,0.672892,0.946403,0.428879,1


In [3]:
data

,X1,X2,X3,X4,X5,X6,X7,class
0,0.217707,0.634695,0.028377,0.093866,0.170345,0.218078,0.254370,0
1,0.976832,0.196437,0.593828,0.225760,0.810395,0.215981,0.463485,1
2,0.474112,0.867036,0.786070,0.634799,0.562943,0.470834,0.642650,1
3,0.435562,0.559760,0.791351,0.891573,0.853098,0.609158,0.281369,1
4,0.497564,0.188632,0.208090,0.960607,0.935006,0.767656,0.707381,1
...,...,...,...,...,...,...,...,...
495,0.843427,0.707178,0.868424,0.785170,0.103390,0.628541,0.669493,1
496,0.922892,0.375219,0.740096,0.045435,0.638138,0.207538,0.697541,0
497,0.548765,0.478041,0.862437,0.822143,0.639047,0.554049,0.857795,1
498,0.251716,0.348931,0.817141,0.137908,0.672892,0.946403,0.428879,1


In [8]:
X_train = data.drop(columns='class', axis=1)
y_train = data['class']

In [9]:
X_test = test_data

In [10]:
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test)

In [11]:
param = {'objective': 'binary:logistic', 'eval_metric': 'auc'}

# 5-fold cross-validation
cv_results = xgb.cv(dtrain=dtrain, params=param, nfold=5, num_boost_round=200, early_stopping_rounds=10, seed=0, metrics='auc', as_pandas=False)

In [13]:
num_boost_rounds = len(cv_results['train-auc-mean'])

bst = xgb.train(param, dtrain, num_boost_round=num_boost_rounds)

prediction = bst.predict(dtest)

In [14]:
rf = RandomForestClassifier(random_state=0)
rf.fit(X_train, y_train)


RandomForestClassifier(random_state=0)

In [15]:
rf_most_important = np.argmax(rf.feature_importances_) + 1

In [16]:
np.savez('submission.npz', rf_most_important=rf_most_important, prediction=prediction)

In [17]:
npzfile = np.load('submission.npz')
print(npzfile['rf_most_important'], npzfile['prediction'].shape)

6 (500,)
